Name: Amirreza Veale

Student ID: 400102222

In this assignment, we will implement and test REINFORCE and PPO, which are both on-policy RL algortihms.

# REINFORCE algorithm **(40 points)**

## Setup

We must first install the required packages.

In [ ]:
!pip -q install gymnasium[mujoco]
!pip -q install gymnasium[classic-control]
!pip -q install imageio==2.19.3
!pip -q install imageio-ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 27.3 MB/s eta 0:00:00


In [ ]:
import gymnasium as gym
import random
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
from collections import namedtuple, deque
import imageio

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal
from torch.distributions.multivariate_normal import MultivariateNormal
from torch.distributions import Categorical

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

## Explore the environment

We will train an REINFORCE agent on the `CartPole` environment.

This code displays a video given it's path.

In [ ]:
from IPython.display import HTML
from base64 import b64encode

def show_video(path):
    mp4 = open(path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=400 controls>
          <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

Explore the `CartPole` environment using random actions. At each timestep, render the current frame, and use it to make a video of the trajectory.

In [ ]:
env = gym.make("CartPole-v1", render_mode="rgb_array")
frames = []

env.reset()
for _ in range(100):
    frames.append(env.render())
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        break
env.close()
imageio.mimsave('./CartPole.mp4', frames, fps=25)
show_video('./CartPole.mp4')


## Policy Network **(10 points)**

Complete the following code to build an agent that predicts the the probability of playing each action, given the state.

In [ ]:
class PolicyNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PolicyNetwork, self).__init__()
        # TODO
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # TODO
        x = F.relu(self.fc1(x))
        action_probs = F.softmax(self.fc2(x), dim=-1)
        return action_probs

## Agent **(20 points)**

REINFORCE algorithm works by interacting with an environment by taking actions based on a policy. As the agent collects rewards from the environment, it records the outcomes and the **log probabilities** of the actions it took. At the end of an episode, the algorithm calculates the total **discounted reward** from each step—this is known as the return.

$$ R_t = \sum_{k=t}^{T} \gamma^{k-t} r_k
 $$

These returns are used to weight the logged probabilities, actions that lead to higher returns are made more probable.


$$ \theta \leftarrow \theta + \alpha \sum_{t=0}^{T-1} \gamma^t R_t \nabla_\theta \log \pi_\theta(a_t|s_t)
 $$


In [ ]:
class REINFORCEAgent:
    def __init__(self, policy, optimizer, gamma=0.99):
        self.policy = policy
        self.optimizer = optimizer
        self.gamma = gamma
        self.log_probs = []
        self.rewards = []

    def select_action(self, state):
        # TODO
        # select an action by sampling from the actor's response
        action_probs = self.policy(state)
        action_distribution = Categorical(action_probs)
        action = action_distribution.sample()
        self.log_probs.append(action_distribution.log_prob(action))
        return action

    def update_policy(self):
        R = 0
        policy_loss = []
        returns = []

        # TODO
        # Calculate the discounted reward
        for r in self.rewards[::-1]:
            R = r + self.gamma * R
            returns.insert(0, R)

        returns = torch.tensor(returns)

        # TODO
        # Calculate the policy loss
        for log_prob, R in zip(self.log_probs, returns):
            policy_loss.append(torch.Tensor(-log_prob * R))


        self.optimizer.zero_grad()
        policy_loss = torch.stack(policy_loss).sum()
        policy_loss.backward()
        self.optimizer.step()

        # Reset the rewards and log probabilities
        del self.rewards[:]
        del self.log_probs[:]

    def store_reward(self, reward):
        self.rewards.append(reward)


## Training **(5 points)**

Define the hyperparameters and complete the training loop.

In [ ]:
env = gym.make('CartPole-v1')
input_size = env.observation_space.shape[0]
output_size = env.action_space.n
lr = 0.001

policy = PolicyNetwork(input_size, 128, output_size).to(device)
optimizer = optim.Adam(policy.parameters(), lr=lr)
agent = REINFORCEAgent(policy, optimizer)

num_episodes = 1000

for episode in range(num_episodes):
    state, info = env.reset()
    total_reward = 0

    # TODO
    # collect rewards and log probabilities for updating the policy in a loop
    while True:
        state = torch.FloatTensor(state).to(device)
        action = agent.select_action(state)
        state, reward, terminated, truncated, info = env.step(action.item())
        done = terminated or truncated
        total_reward += reward
        agent.store_reward(reward)
        if done:
            break

    agent.update_policy()
    if episode % 50 == 0:
        print(f'Episode {episode+1}: Total Reward = {total_reward}')
env.close()

Episode 1: Total Reward = 26.0
Episode 51: Total Reward = 19.0
Episode 101: Total Reward = 35.0
Episode 151: Total Reward = 299.0
Episode 201: Total Reward = 65.0
Episode 251: Total Reward = 52.0
Episode 301: Total Reward = 230.0
Episode 351: Total Reward = 161.0
Episode 401: Total Reward = 228.0
Episode 451: Total Reward = 388.0
Episode 501: Total Reward = 173.0
Episode 551: Total Reward = 264.0
Episode 601: Total Reward = 241.0
Episode 651: Total Reward = 295.0
Episode 701: Total Reward = 500.0
Episode 751: Total Reward = 375.0
Episode 801: Total Reward = 180.0
Episode 851: Total Reward = 500.0
Episode 901: Total Reward = 500.0
Episode 951: Total Reward = 500.0


## Evaluation **(5 points)**

Here we use the trained agent and collect a trajectory using it's policy. Calculate the cumulative reward by adding rewards in each time space. Save and display the video of this run in the end.

In [ ]:
env = gym.make("CartPole-v1", render_mode="rgb_array")
state, _ = env.reset()
frames = []

total_reward = 0
# TODO
# run the policy in the environment in a loop
while True:
    frames.append(env.render())
    state = torch.FloatTensor(state).to(device)
    action = agent.select_action(state)
    state, reward, terminated, truncated, info = env.step(action.item())
    done = terminated or truncated
    total_reward += reward
    if done:
        break

env.close()
print(f'Total Reward: {total_reward}')

imageio.mimsave('./eval_reinforce.mp4', frames, fps=25)
show_video('./eval_reinforce.mp4')

Total Reward: 500.0


# Proximal Policy Optimization **(60 points)**

## Setup

## Explore the environment

This code is essential for rendering MUJOCO based environments.

In [ ]:
# Configure MuJoCo to use the EGL rendering backend (requires GPU)
%env MUJOCO_GL=egl

env: MUJOCO_GL=egl


We will train a PPO agent in the `HalfCheetah` environment. This environment features continuous actions and more complex mechanics.

Explore this environment using random actions as well, and display the video of the resulting trajectory.

* What are the observation and action spaces of this environment?

* Are values bounded?

In [ ]:
env = gym.make("HalfCheetah-v4", render_mode="rgb_array")
env.reset()
frames = []

for _ in range(100):
    frames.append(env.render())
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        break
env.close()
imageio.mimsave('./HalfCheetah.mp4', frames, fps=25)
show_video('./HalfCheetah.mp4')


## Actor & Critic **(15 points)**

Proximal Policy Optimization (PPO) is an advanced reinforcement learning algorithm that uses separate actor and critic networks to optimize policy performance.

The actor network is responsible for predicting a probability distribution over actions (discrete) or estimating the value for each action (continuous), given the current state, while the critic network evaluates how good the action taken by the actor is, by predicting the reward based on state.


In [ ]:
class Actor(nn.Module):
    def __init__(self, state_dim, hidden_size, action_dim):
        super(Actor, self).__init__()
        # TODO
        # Define the Actor architecture
        self.fc_mu_1 = nn.Linear(state_dim, hidden_size)
        self.fc_mu_2 = nn.Linear(hidden_size, hidden_size)
        self.fc_mu_3 = nn.Linear(hidden_size, action_dim)

        self.fc_std_1 = nn.Linear(state_dim, hidden_size)
        self.fc_std_2 = nn.Linear(hidden_size, hidden_size)
        self.fc_std_3 = nn.Linear(hidden_size, action_dim)



    def forward(self, state):
        # TODO
        # In case of continuous environment, we usually
        # predict a mean and std for each action and sample
        # the action from a normal distribution
        mu = F.relu(self.fc_mu_1(state))
        mu = F.relu(self.fc_mu_2(mu))
        mu = self.fc_mu_3(mu)

        std = F.relu(self.fc_std_1(state))
        std = F.relu(self.fc_std_2(std))
        std = self.fc_std_3(std)
        std = F.softplus(std) + 1e-5
        return mu, std

class Critic(nn.Module):
    def __init__(self, state_dim, hidden_size):
        super(Critic, self).__init__()
        # TODO
        # Define the Critic architecture
        self.fc1 = nn.Linear(state_dim, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 1)

    def forward(self, state):
        # TODO
        # Predict the value of state
        value = F.relu(self.fc1(state))
        value = F.relu(self.fc2(value))
        value = self.fc3(value)
        return value


## Memory

PPO algorithms need to store sequences of actions, states, log probabilities, rewards, and state values to train the agent. This data is captured in the `Memory` class, which facilitates batch processing by holding and then clearing these elements at the end of each training iteration.

In [ ]:
class Memory:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.state_values = []

    def clear(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.state_values[:]


## Agent **(35 points)**

In PPO, the actor's goal is to maximize the expected return. However, direct maximization can cause large policy updates, risking instability. To prevent this, PPO employs a clipping mechanism, limiting policy changes to a defined range.

$$ L^{CLIP}(\theta) = \hat{\mathbb{E}}_t \left[ \min(r_t(\theta) \hat{A}_t, \text{clip}(r_t(\theta), 1-\epsilon, 1+\epsilon) \hat{A}_t) \right]
 $$

Additionally, it uses a probability ratio to scale updates, ensuring changes This ratio provides a scaling factor for the policy updates, ensuring that changes are made in proportion to the improvement in policy performance.

$$ r_t(\theta) = \frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_{old}}(a_t|s_t)}
 $$

 The critic aims to minimize the error between its predictions and the actual returns.

 $$ L^{VF}(\phi) = \left( V_\phi(s_t) - \hat{R}_t \right)^2
 $$

In [ ]:
class PPO(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_size=64, lr_actor=1e-4, lr_critic=1e-4, gamma=0.99, epochs=4, eps_clip=0.2):
        super(PPO, self).__init__()
        self.gamma = gamma
        self.eps_clip = eps_clip
        self.epochs = epochs

        self.actor = Actor(state_dim, hidden_size, action_dim)
        self.critic = Critic(state_dim, hidden_size)

        self.optimizer_actor = optim.Adam(self.actor.parameters(), lr=lr_actor)
        self.optimizer_critic = optim.Adam(self.critic.parameters(), lr=lr_critic)
        self.memory = Memory()

    def select_action(self, state):
        # TODO
        # Save state, action, log probability and state value of current step in the memory buffer.
        # predict the actions by sampling from a normal distribution
        # based on the mean and std calculated by actor
        state = torch.FloatTensor(state).to(device)
        mu, std = self.actor(state)
        action_distribution = MultivariateNormal(mu, torch.diag_embed(std))
        action = action_distribution.sample()
        self.memory.states.append(state)
        self.memory.actions.append(action)
        self.memory.logprobs.append(action_distribution.log_prob(action))
        self.memory.state_values.append(self.critic(state))
        return action

    def evaluate(self, state, action):
        # TODO
        # evaluate the state value of this state and log probability of choosing this action
        mu, std = self.actor(state)
        action_distribution = MultivariateNormal(mu, torch.diag_embed(std))
        action_logprobs = action_distribution.log_prob(action)
        state_value = self.critic(state)
        entropy = action_distribution.entropy()
        return action_logprobs, state_value, entropy

    def update(self):
        rewards = []
        discounted_reward = 0
        # TODO
        # Calculate discounted rewards
        for reward in reversed(self.memory.rewards):
            discounted_reward = reward + self.gamma * discounted_reward
            rewards.insert(0, discounted_reward)


        # TODO
        # load saved states, actions, log probs, and state values
        states = torch.stack(self.memory.states).to(device).detach()
        actions = torch.stack(self.memory.actions).to(device).detach()
        logprobs_old = torch.stack(self.memory.logprobs).to(device).detach()
        state_values = torch.stack(self.memory.state_values).to(device).detach()
        rewards = torch.tensor(rewards,dtype = torch.float32).to(device).detach()

        # TODO
        # Calculate advantages for each timestep (usually difference of rewards and state values)
        advantages = rewards - state_values


        loss_ac = 0
        loss_cri = 0
        for _ in range(self.epochs):
            # calculate logprobs and state values based on the new policy
            logprobs, state_values, entropy = self.evaluate(torch.cat(self.memory.states).view(-1,17).to(device) ,torch.cat(self.memory.actions).view(-1,6).to(device))

            # TODO
            # Calculate the loss function and perform the optimization
            ratio = torch.exp(logprobs - logprobs_old)
            surr1 = ratio * advantages
            surr2 = torch.clamp(ratio, 1-self.eps_clip, 1+self.eps_clip) * advantages
            # print(surr1.shape)
            # print(surr2.shape)
            loss_actor = -torch.min(surr1, surr2).mean() - 0.01 * entropy.mean()
            # print(state_values.view(-1).shape)
            loss_critic = 0.5 * F.mse_loss(state_values.view(-1), rewards).view(-1, 1)


            self.optimizer_actor.zero_grad()
            loss_actor.backward()
            loss_ac += loss_actor.item()
            self.optimizer_actor.step()

            self.optimizer_critic.zero_grad()
            loss_critic.backward()
            loss_cri += loss_critic.item()
            self.optimizer_critic.step()

        # clear the buffer
        self.memory.clear()
        return loss_ac, loss_cri

## Training **(5 points)**

Define the hyperparameters and complete the training loop.

In [ ]:
env = gym.make("HalfCheetah-v4")
state_dim = env.observation_space.shape[0]
action_dim = 6
hidden_size = 64

agent = model = PPO(state_dim, action_dim, hidden_size=hidden_size, lr_actor=3e-4, lr_critic=1e-3).to(device)

# We need to train for many more steps to achieve acceptable results compared to the last environment
num_episodes = 6000

actor_losses = []
critic_losses = []
moving_rewards = np.array([])

for episode in range(num_episodes):
    state, _ = env.reset()
    total_reward = 0
    # TODO
    # write the training loop
    while True:
        action = agent.select_action(state)
        next_state, reward, terminated, truncated, info = env.step(action.cpu().numpy())
        agent.memory.rewards.append(reward)
        reward = torch.tensor(reward, dtype=torch.float32, device=device).unsqueeze(0)
        state = next_state
        total_reward += reward
        done = truncated or terminated
        if done:
            break

    loss_ac, loss_cri = agent.update()
    actor_losses.append(loss_ac)
    critic_losses.append(loss_cri)
    moving_rewards = np.append(moving_rewards, total_reward.cpu())
    if episode % 100 == 0:
        print(f"actor loss:\t{loss_ac:.6f}")
        print(f"critic loss:\t{loss_cri:.6f}")
        print(f'Episode {episode}: Going Reward = {moving_rewards.mean():.1f}: Std = {moving_rewards.std():.1f}')
        moving_rewards = np.array([])

env.close()

actor loss:	126.994545
critic loss:	2531.274780
Episode 0: Going Reward = -347.5: Std = 0.0
actor loss:	16.862791
critic loss:	373.714745
Episode 100: Going Reward = -374.8: Std = 103.1
actor loss:	-48.632726
critic loss:	485.819374
Episode 200: Going Reward = -232.5: Std = 149.1
actor loss:	-22.061759
critic loss:	387.358437
Episode 300: Going Reward = 5.4: Std = 245.0
actor loss:	-89.985212
critic loss:	2037.138428
Episode 400: Going Reward = 131.2: Std = 257.2
actor loss:	14.355077
critic loss:	999.508972
Episode 500: Going Reward = 430.9: Std = 285.1
actor loss:	63.891518
critic loss:	1836.431946
Episode 600: Going Reward = 690.1: Std = 178.2
actor loss:	-19.070501
critic loss:	1930.284973
Episode 700: Going Reward = 994.4: Std = 111.0
actor loss:	-15.449124
critic loss:	1052.404297
Episode 800: Going Reward = 1015.7: Std = 85.3
actor loss:	28.220660
critic loss:	1864.869843
Episode 900: Going Reward = 1027.5: Std = 98.0
actor loss:	306.620987
critic loss:	14269.077881
Episode 1000

## Evaluation **(5 points)**

Evaluate the trained policy on the environment. Calculate the cumulative reward and display the video of the trajectory.

In [ ]:
env = gym.make("HalfCheetah-v4", render_mode="rgb_array")
state, _ = env.reset()
frames = []

total_reward = 0
# TODO
# run the policy in the environment in a loop
while True:
    frames.append(env.render())
    action = agent.select_action(state)
    next_state, reward, terminated, truncated, info = env.step(action.cpu().numpy())
    agent.memory.rewards.append(reward)
    reward = torch.tensor(reward, dtype=torch.float32, device=device).unsqueeze(0)
    state = next_state
    total_reward += reward
    done = truncated or terminated
    if done:
        break

env.close()
print(f'Total Reward: {total_reward}')

imageio.mimsave('./eval_ppo.mp4', frames, fps=25)
show_video('./eval_ppo.mp4')

Total Reward: tensor([2077.5283])
